In [4]:
#读取总数据表
import numpy as np
import pandas as pd

real_data=pd.read_csv(r'C:\Users\ljs11\Desktop\实验设计作业\Final_code\data\real_data.csv')
########################################################################################################################

In [5]:
#开始模型运算
import os
import numpy as np
import pandas as pd
# from parameter import *
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
# from run import *

print('开始模型运算')

# 删除某一类别占比超过99%的列
good_cols = list(real_data.columns)
for col in real_data.columns:
    rate = real_data[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.999:
        good_cols.remove(col)
        print(col, rate)

real_data = real_data[good_cols]

train = real_data[real_data['TrainOrTest'] == 1]
test = real_data[real_data['TrainOrTest'] == 0]

submission = pd.DataFrame()
submission['id'] = test['id']

params_lgb = {
        'boosting_type': 'gbdt',
        'n_estimators': 1000,
        'max_depth': -1,
        'objective': 'binary',
        'metric': {'auc'},
        'num_leaves': 16,
        'learning_rate': 0.1,
        'feature_fraction': 1.,
        'bagging_fraction': 1.,
        'reg_lambda': 0.5,
        'reg_alpha': 0.3,
        'random_state': 1024,
        'n_jobs': -1,
    }

drop_fea = ['id', 'flag', 'isTest','TrainOrTest']
    # features = [col for col in test.columns if col not in drop_fea]
    # TOP100
features = ['cur_credit_min_opn_dt_cnt', 'l1y_crd_card_csm_amt_dlm_cd', 'Dat_Flg1_Cd_per_count_amt_1',
                'perm_crd_lmt_cd',
                'Trx_Cod2_Cd54_recent_trd',
                # 'cny_trx_amt_sum_6',
                'Dat_Flg3_Cd1_recent_trd', 'acdm_deg_cd',
                'cur_debit_min_opn_dt_cnt', 'Dat_Flg3_Cd_per_count_amt_1', 'Trx_Cod1_Cd_amt_2',
                'Dat_Flg1_Cd_per_count_amt_0',
                'Trx_Cod1_Cd_per_count_amt_2', 'Trx_Cod1_Cd_per_day_times_2', 'Trx_Cod1_Cd_amt_0', 'gdr_cd',
                'Trx_Cod2_Cd_per_count_amt_36', 'pot_ast_lvl_cd', 'job_year',
                # 'cny_trx_amt_sum_5',
                'hld_crd_card_grd_cd',
                'Trx_Cod1_Cd_per_count_amt_0', 'fr_or_sh_ind', 'age', '6_Dat_Flg3_Cd_times_1',
                'Dat_Flg1_Cd_per_day_times_1',
                'Trx_Cod1_Cd1_recent_trd', 'Trx_Cod2_Cd_per_day_times_2', 'Trx_Cod2_Cd_amt_36',
                'Dat_Flg1_Cd5_per_day_times_0',
                'per_day_trd_amt', 'Trx_Cod2_Cd53_recent_trd', 'Trx_Cod2_Cd5_per_day_times_26', 'his_lng_ovd_day',
                'hav_car_grp_ind', 'Dat_Flg1_Cd5_per_day_times_1',
                # 'Trx_Cod2_Cd1_recent_trd',
                'Dat_Flg3_Cd_per_count_amt_0',
                # 'trd_amt',
                'dnl_bind_cmb_lif_ind', 'Trx_Cod2_Cd6_per_day_times_32', 'Trx_Cod2_Cd6_per_day_times_54',
                'Trx_Cod2_Cd_amt_54', 'Dat_Flg1_Cd6_per_day_times_1', 'Trx_Cod2_Cd_per_count_amt_30',
                'Trx_Cod2_Cd6_per_day_times_2', 'Dat_Flg3_Cd_amt_1', 'per_count_trd_amt',
                'Trx_Cod2_Cd_per_day_times_46',
                'Dat_Flg3_Cd6_per_day_times_1',
                # 'Trx_Cod1_Cd2_recent_trd',
                'frs_agn_dt_cnt', 'edu_deg_cd',
                'Trx_Cod2_Cd_per_day_times_15', 'dnl_mbl_bnk_ind', 'Trx_Cod2_Cd29_recent_trd', 'Dat_Flg1_Cd_amt_1',
                'Trx_Cod2_Cd_amt_32', 'Trx_Cod2_Cd_per_count_amt_55', 'crd_card_act_ind',
                'Trx_Cod2_Cd_per_count_amt_32',
                'Dat_Flg3_Cd_per_day_times_1', 'Trx_Cod2_Cd_amt_53', 'Trx_Cod2_Cd32_recent_trd', 'Dat_Flg1_Cd_amt_0',
                'Trx_Cod2_Cd5_per_day_times_32', 'per_day_trd_times', 'Trx_Cod2_Cd_amt_55', 'Trx_Cod2_Cd_amt_30',
                'Trx_Cod2_Cd_amt_52', '5_Trx_Cod2_Cd_times_54', 'Trx_Cod2_Cd_per_day_times_40',
                'Trx_Cod2_Cd_per_count_amt_54',
                'Trx_Cod2_Cd_per_day_times_3', 'Trx_Cod1_Cd5_per_day_times_2', 'Dat_Flg3_Cd_times_1',
                'Trx_Cod2_Cd_per_day_times_54', 'Dat_Flg3_Cd5_per_day_times_1', '6_Trx_Cod2_Cd_times_10',
                'Trx_Cod2_Cd6_per_day_times_1', 'Trx_Cod2_Cd5_per_day_times_30', 'Trx_Cod2_Cd_per_count_amt_11',
                'Trx_Cod2_Cd_amt_24', 'Dat_Flg3_Cd_amt_0', 'Trx_Cod1_Cd6_per_day_times_2',
                # 'Trx_Cod2_Cd30_recent_trd',
                'Trx_Cod2_Cd_per_day_times_16', '6_Trx_Cod2_Cd_times_54', 'Trx_Cod2_Cd6_per_day_times_55',
                'Trx_Cod2_Cd_per_day_times_32',
                # 'Trx_Cod1_Cd0_recent_trd',
                'Trx_Cod2_Cd_per_day_times_0',
                'Trx_Cod1_Cd_per_day_times_1', 'Trx_Cod1_Cd_per_count_amt_1', 'Trx_Cod2_Cd_per_count_amt_21',
                'Trx_Cod2_Cd_times_54', 'Dat_Flg1_Cd_per_day_times_0', 'Trx_Cod2_Cd5_per_day_times_54',
                # 'Trx_Cod2_Cd16_recent_trd',
                'Dat_Flg1_Cd0_recent_trd']

train_x = train[features]
train_y = train['flag']
test_x = test[features]
test_y = test['flag']

##################################################################################################################

开始模型运算
isTest 1.0
Trx_Cod2_Cd_times_20 1.0
Trx_Cod2_Cd_times_43 0.9999749517821807
Trx_Cod2_Cd_times_56 0.9993487463366981
Trx_Cod2_Cd_per_day_times_20 1.0
Trx_Cod2_Cd_per_day_times_43 0.9999749517821807
Trx_Cod2_Cd_per_day_times_56 0.9993487463366981
Trx_Cod2_Cd_amt_20 1.0
Trx_Cod2_Cd_amt_43 0.9999749517821807
Trx_Cod2_Cd_amt_56 0.999423890990156
Trx_Cod2_Cd_per_count_amt_20 1.0
Trx_Cod2_Cd_per_count_amt_43 0.9999749517821807
Trx_Cod2_Cd_per_count_amt_56 0.999423890990156
5_Trx_Cod2_Cd_times_20 1.0
5_Trx_Cod2_Cd_times_44 0.9995741802970719
5_Trx_Cod2_Cd_times_50 0.9991483605941437
5_Trx_Cod2_Cd_times_56 0.999499035643614
Trx_Cod2_Cd5_per_day_times_20 1.0
Trx_Cod2_Cd5_per_day_times_44 0.9995741802970719
Trx_Cod2_Cd5_per_day_times_50 0.9991483605941437
Trx_Cod2_Cd5_per_day_times_56 0.999499035643614
6_Trx_Cod2_Cd_times_20 1.0
6_Trx_Cod2_Cd_times_43 0.9999749517821807
6_Trx_Cod2_Cd_times_50 0.999173408811963
6_Trx_Cod2_Cd_times_56 0.9996994213861684
Trx_Cod2_Cd6_per_day_times_20 1.0
Trx_

In [6]:
#神经网络MLPClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
clf = MLPClassifier(solver='lbfgs', alpha=1e-5 , hidden_layer_sizes=(5, 2), random_state=1)

#划分交叉训练数据集
# SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)#5折交叉验证
# SKF = StratifiedKFold(n_splits=10, shuffle=True, random_state=2019)#10折交叉验证
SKF = StratifiedKFold(n_splits=20, shuffle=True, random_state=2019)#20折交叉验证
# SKF = StratifiedKFold(n_splits=30, shuffle=True, random_state=2019)#30折交叉验证

train_X=train_x.values
train_Y=train_y.values
for train, test in SKF.split(train_X, train_Y):
    clf.fit(train_X[train], train_Y[train])#训练，使用标准化降维后x
    y_test_predict = clf.predict_proba(test_x)[:,1]

    # 返回预测属于某标签的概率
    print(clf.predict_proba(test_x))
    # print(classification_report(test_y, y_test_predict))#输出模型评价标准

print('结束')

#返回模型评价指标
# print(classification_report(test_y, y_test_predict))#输出模型评价标准

submission['pred'] = y_test_predict
submission.to_csv(r'C:\Users\ljs11\Desktop\实验设计作业\Final_code\data\Result\神经网络_结果.csv', index=False, header=0, encoding='utf-8')

##################################################################################################################

[[0.48521971 0.51478029]
 [0.48521971 0.51478029]
 [0.48521971 0.51478029]
 ...
 [0.48521971 0.51478029]
 [0.59186954 0.40813046]
 [0.48521971 0.51478029]]
[[0.4855828  0.5144172 ]
 [0.4855828  0.5144172 ]
 [0.4855828  0.5144172 ]
 ...
 [0.4855828  0.5144172 ]
 [0.59108959 0.40891041]
 [0.4855828  0.5144172 ]]
[[0.48557133 0.51442867]
 [0.48557133 0.51442867]
 [0.48557133 0.51442867]
 ...
 [0.48557133 0.51442867]
 [0.59228174 0.40771826]
 [0.48557133 0.51442867]]
[[0.48559046 0.51440954]
 [0.48559046 0.51440954]
 [0.48559046 0.51440954]
 ...
 [0.48559046 0.51440954]
 [0.59237953 0.40762047]
 [0.48559046 0.51440954]]
[[0.48574373 0.51425627]
 [0.48574373 0.51425627]
 [0.48574373 0.51425627]
 ...
 [0.48574373 0.51425627]
 [0.59237417 0.40762583]
 [0.48574373 0.51425627]]
[[0.48801932 0.51198068]
 [0.48801932 0.51198068]
 [0.48801932 0.51198068]
 ...
 [0.48801932 0.51198068]
 [0.59426347 0.40573653]
 [0.48801932 0.51198068]]
[[0.48565581 0.51434419]
 [0.48565581 0.51434419]
 [0.48565581 0